### 1. Year-Over-Year Sales Comparison

**Functional Specification**
- Group sales data by Year from Sales.SalesOrderHeader.
- Sum the TotalDue for each year.
- Use LAG() to retrieve the sum of sales from the previous year (Offset 1).
- Order the window by Year.
- Calculate the difference between Current Year Sales and Previous Year Sales.

In [ ]:
WITH YearlySales AS (
    SELECT 
        YEAR(OrderDate) AS SalesYear,
        SUM(TotalDue) AS TotalRevenue
    FROM Sales.SalesOrderHeader
    GROUP BY YEAR(OrderDate)
)
SELECT 
    SalesYear,
    TotalRevenue,
    LAG(TotalRevenue, 1, 0) OVER (ORDER BY SalesYear) AS PrevYearRevenue,
    TotalRevenue - LAG(TotalRevenue, 1, 0) OVER (ORDER BY SalesYear) AS YoYGrowth
FROM YearlySales;

### 2. Days Since Last Order per Customer

**Functional Specification**
- Select CustomerID and OrderDate from Sales.SalesOrderHeader.
- Use LAG() to find the OrderDate of the row immediately preceding the current one.
- Partition by CustomerID so we only look at that specific customer's history.
- Order by OrderDate ASC.
- Use DATEDIFF to calculate days between the current order and the lagged order date.

In [ ]:
SELECT 
    SalesOrderID,
    CustomerID,
    OrderDate,
    LAG(OrderDate) OVER (PARTITION BY CustomerID ORDER BY OrderDate) AS PreviousOrderDate,
    DATEDIFF(day, LAG(OrderDate) OVER (PARTITION BY CustomerID ORDER BY OrderDate), OrderDate) AS DaysSinceLastOrder
FROM Sales.SalesOrderHeader;

### 3. Compare Sales Quota to Next Quota

**Functional Specification**
- Select BusinessEntityID, QuotaDate, and SalesQuota from Sales.SalesPersonQuotaHistory.
- Use LEAD() to peek at the next chronological quota record for that salesperson.
- Partition by BusinessEntityID.
- Order by QuotaDate.
- Return current quota, next quota, and the date of the change.

In [ ]:
SELECT 
    BusinessEntityID,
    QuotaDate,
    SalesQuota,
    LEAD(SalesQuota) OVER (PARTITION BY BusinessEntityID ORDER BY QuotaDate) AS NextQuota,
    LEAD(SalesQuota) OVER (PARTITION BY BusinessEntityID ORDER BY QuotaDate) - SalesQuota AS QuotaChange
FROM Sales.SalesPersonQuotaHistory;

### 4. First Order Value vs Current Order

**Functional Specification**
- Select order details for customers.
- Use FIRST_VALUE() to retrieve the TotalDue of the very first order the customer placed.
- Partition by CustomerID.
- Order by OrderDate.
- Allow comparison between the current order's value and their initial purchase size.

In [ ]:
SELECT 
    CustomerID,
    SalesOrderID,
    OrderDate,
    TotalDue,
    FIRST_VALUE(TotalDue) OVER (PARTITION BY CustomerID ORDER BY OrderDate) AS FirstOrderAmount
FROM Sales.SalesOrderHeader;

### 5. Last Recorded Price in Price History

**Functional Specification**
- Use Production.ProductListPriceHistory.
- Use LAST_VALUE() to find the most recent list price recorded within a window.
- Partition by ProductID.
- Order by StartDate.
- Note: LAST_VALUE requires explicit framing (ROWS BETWEEN ...) to work as expected for "latest in dataset" vs "latest so far".

In [ ]:
SELECT 
    ProductID,
    StartDate,
    ListPrice,
    LAST_VALUE(ListPrice) OVER ( 
        PARTITION BY ProductID 
        ORDER BY StartDate 
        ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING
    ) AS MostRecentPrice
FROM Production.ProductListPriceHistory;

### 6. Employee Pay Rate Changes

**Functional Specification**
- Query HumanResources.EmployeePayHistory.
- Use LAG() to compare the current Rate with the previous Rate.
- Partition by BusinessEntityID (Employee).
- Order by RateChangeDate.
- Calculate the percentage increase in pay.

In [ ]:
SELECT 
    BusinessEntityID,
    RateChangeDate,
    Rate,
    LAG(Rate) OVER (PARTITION BY BusinessEntityID ORDER BY RateChangeDate) AS PreviousRate,
    (Rate - LAG(Rate) OVER (PARTITION BY BusinessEntityID ORDER BY RateChangeDate)) / 
    LAG(Rate) OVER (PARTITION BY BusinessEntityID ORDER BY RateChangeDate) * 100 AS PercentIncrease
FROM HumanResources.EmployeePayHistory;

### 7. Inventory Location Movement

**Functional Specification**
- This is a conceptual proposition assuming tracking movement (or simply ordering inventory by location ID for analysis).
- Query Production.ProductInventory.
- Partition by ProductID.
- Order by LocationID (assuming movement flow from Loc 1 to Loc 60).
- Use LEAD() to see the quantity in the next stage of the production line.

In [ ]:
SELECT 
    ProductID,
    LocationID,
    Quantity,
    LEAD(Quantity) OVER (PARTITION BY ProductID ORDER BY LocationID) AS NextLocationQuantity
FROM Production.ProductInventory;

### 8. Previous Order ID for Gap Analysis

**Functional Specification**
- Select SalesOrderID from Sales.SalesOrderHeader.
- Order by SalesOrderID.
- Use LAG() to get the previous SalesOrderID.
- Calculate if there are gaps in the ID sequence (Current ID - Prev ID > 1).

In [ ]:
WITH OrderGaps AS (
    SELECT 
        SalesOrderID,
        LAG(SalesOrderID) OVER (ORDER BY SalesOrderID) AS PrevOrderID,
        SalesOrderID - LAG(SalesOrderID) OVER (ORDER BY SalesOrderID) AS IDDiff
    FROM Sales.SalesOrderHeader
)
SELECT *
FROM OrderGaps
WHERE IDDiff > 1;

### 9. Standard Cost Trends

**Functional Specification**
- Query Production.ProductCostHistory.
- Use LAG() to pull the previous StandardCost.
- Partition by ProductID.
- Order by StartDate.
- Determine if costs are rising or falling.

In [ ]:
SELECT 
    ProductID,
    StartDate,
    StandardCost,
    LAG(StandardCost) OVER (PARTITION BY ProductID ORDER BY StartDate) AS PreviousCost,
    CASE 
        WHEN StandardCost > LAG(StandardCost) OVER (PARTITION BY ProductID ORDER BY StartDate) THEN 'Increased'
        WHEN StandardCost < LAG(StandardCost) OVER (PARTITION BY ProductID ORDER BY StartDate) THEN 'Decreased'
        ELSE 'No Change' 
    END AS Trend
FROM Production.ProductCostHistory;

### 10. Next Shift for Employee

**Functional Specification**
- Join HumanResources.EmployeeDepartmentHistory and HumanResources.Shift.
- (Note: AdventureWorks shift history is limited, so we will simulate logical shifts based on Dept history).
- Use LEAD() to identify the department an employee moved to after their current one.
- Partition by BusinessEntityID.
- Order by StartDate.

In [ ]:
SELECT 
    BusinessEntityID,
    DepartmentID,
    StartDate,
    LEAD(DepartmentID) OVER (PARTITION BY BusinessEntityID ORDER BY StartDate) AS NextDepartmentID
FROM HumanResources.EmployeeDepartmentHistory;